In [ ]:
%load_ext autoreload
%autoreload 2
import helper_methods_for_aggregate_data_analysis as helper
import numpy as np
import pandas as pd
from disease_model import * 
from model_experiments import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker as tick
from collections import Counter 
import datetime
import os
import pickle

In [ ]:
MIN_DATETIME = datetime.datetime(2020, 3, 1, 0)
MAX_DATETIME = datetime.datetime(2020, 5, 2, 23)
MSAS = ['Atlanta_Sandy_Springs_Roswell_GA',
        'Chicago_Naperville_Elgin_IL_IN_WI',
        'Dallas_Fort_Worth_Arlington_TX',
        'Houston_The_Woodlands_Sugar_Land_TX',
        'Los_Angeles_Long_Beach_Anaheim_CA',
        'Miami_Fort_Lauderdale_West_Palm_Beach_FL',
        'New_York_Newark_Jersey_City_NY_NJ_PA',
        'Philadelphia_Camden_Wilmington_PA_NJ_DE_MD',
        'San_Francisco_Oakland_Hayward_CA',
        'Washington_Arlington_Alexandria_DC_VA_MD_WV']
DC_NAME = 'Washington_Arlington_Alexandria_DC_VA_MD_WV'

LOWER_PERCENTILE = 2.5
UPPER_PERCENTILE = 97.5
INCIDENCE_POP = 100000

# Model fit 

In [ ]:
msa_to_highlight = 'Washington_Arlington_Alexandria_DC_VA_MD_WV'
msa_pretty_name = MSAS_TO_PRETTY_NAMES[msa_to_highlight]

## Load models

In [ ]:
MIN_TIMESTRING_FOR_GRID_SEARCH = '2020_05_23_13_50_41_383327'
all_results = []
for msa in MSAS:
    all_results.append(
        evaluate_all_fitted_models_for_msa(
            msa, 
            min_timestring=MIN_TIMESTRING_FOR_GRID_SEARCH))
df = pd.concat(all_results)
df['MSA_name'] = df['data_kwargs'].map(lambda x:x['MSA_name'])
df['ipf_final_match'] = df['model_init_kwargs'].map(lambda x:x['ipf_final_match'])
df = df.loc[df['experiment_to_run'] == 'normal_grid_search']
print("Filtering for grid search models; total models loaded: %i" % len(df))
ablation_df = df.loc[df['poi_psi'] == 0].copy()
non_ablation_df = df.loc[df['poi_psi'] > 0].copy()
df = None

## Grid search parameter table

In [ ]:
def get_models_over_threshold(df, msas, threshold, max_models):

    keys = ['home_beta', 'poi_psi', 'p_sick_at_t0']
    columns = ['MSA', 'n_models'] + [f'{key}_min' for key in keys] + [f'{key}_max' for key in keys]
    model_df = pd.DataFrame(columns=columns)
    model_df_idx = 0

    for msa_name in msas:
        subdf = df[(df['MSA_name'] == msa_name)].copy()

        loss_key = 'loss_dict_daily_cases_RMSE'
        subdf = subdf.sort_values(by=loss_key)
        losses = subdf[loss_key] / subdf.iloc[0][loss_key]
        n_models = min(max_models, np.sum(losses <= threshold))

        model_dict = {}
        model_dict['MSA'] = msa_name
        # Best fit model
        for key in keys:
            model_dict[key] = subdf.iloc[0][key]

        for key in keys:
            model_dict[f'{key}_max'] = -1000000
            model_dict[f'{key}_min'] =  1000000

        # Mins and maxes
        for subdf_idx in range(n_models):
            for key in keys:
                min_key = f'{key}_min'
                max_key = f'{key}_max'
                if model_dict[min_key] > subdf.iloc[subdf_idx][key]:
                    model_dict[min_key] = subdf.iloc[subdf_idx][key]
                if model_dict[max_key] < subdf.iloc[subdf_idx][key]:
                    model_dict[max_key] = subdf.iloc[subdf_idx][key]

        model_dict['n_models'] = n_models
        model_df = model_df.append(model_dict, ignore_index=True)
        print(f'{msa_name:50s}: {n_models:3d}')

    return model_df

print(f'Loss tolerance: {ACCEPTABLE_LOSS_TOLERANCE}')
model_df = get_models_over_threshold(
    non_ablation_df, 
    MSAS,
    ACCEPTABLE_LOSS_TOLERANCE, 
    MAX_MODELS_TO_TAKE_PER_MSA)
keys = ['home_beta', 'poi_psi', 'p_sick_at_t0']
table_df = pd.DataFrame(columns=['MSA', '# models within RMSE threshold'] + keys)
for idx in range(len(model_df)):
    model_dict = {}
    model_dict['MSA'] = MSAS_TO_PRETTY_NAMES[model_df.iloc[idx]['MSA']]
    model_dict['# models within RMSE threshold'] = model_df.iloc[idx]['n_models']
    for key in keys:
        min_key = f'{key}_min'
        max_key = f'{key}_max'
        model_dict[key] = f'{model_df.iloc[idx][key]} ({model_df.iloc[idx][min_key]}, {model_df.iloc[idx][max_key]})'
    table_df = table_df.append(model_dict, ignore_index=True)
table_df

## Figure 1, model fits

In [ ]:
df = non_ablation_df
thing_to_plot = 'cases'
prefix_to_save_plot_with = f'trajectory_{thing_to_plot}_oos_vs_full_fit_{msa_pretty_name}'
plot_log = False
plot_daily_not_cumulative = False

fig, axes = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True)

for idx, ax in enumerate(axes):
    
    if idx == 0:
        title = 'Out-of-sample fit'
        plot_legend = False
        train_test_partition = TRAIN_TEST_PARTITION
        key_to_sort_by = 'train_loss_dict_daily_cases_RMSE'
    else:
        title = 'Full fit'
        plot_legend = True
        train_test_partition = None
        key_to_sort_by = 'loss_dict_daily_cases_RMSE'
        
    subdf = df[(df['MSA_name'] == msa_to_highlight)].copy()
    subdf = subdf.sort_values(by=key_to_sort_by)    
    losses = subdf[key_to_sort_by] / subdf.iloc[0][key_to_sort_by]
    num_models_to_aggregate = np.sum(losses <= ACCEPTABLE_LOSS_TOLERANCE)
        
    mdl_predictions = []
    old_projected_hrs = None
    for model_idx in range(num_models_to_aggregate):
        ts = subdf.iloc[model_idx]['timestring']
        _, kwargs, _, model_results, _ = load_model_and_data_from_timestring(ts, load_fast_results_only=False)
        model_kwargs = kwargs['model_kwargs']
        data_kwargs = kwargs['data_kwargs']
        mdl_prediction, projected_hrs = plot_model_fit_from_model_and_kwargs(ax, 
                                             model_kwargs, 
                                             data_kwargs, 
                                             model_results=model_results, 
                                             plotting_kwargs={'make_plot':False,
                                                              'plot_daily_not_cumulative':plot_daily_not_cumulative,                                                              
                                                              'return_mdl_pred_and_hours':True}, 
                                             train_test_partition=train_test_partition,
                                         )
        mdl_predictions.append(mdl_prediction)
        if old_projected_hrs is not None:
            assert projected_hrs == old_projected_hrs
        old_projected_hrs = projected_hrs
    mdl_predictions = np.concatenate(mdl_predictions, axis=0)
        
    plot_model_fit_from_model_and_kwargs(
        ax, 
        model_kwargs, 
        data_kwargs,         
        plotting_kwargs={
            'plot_mode':thing_to_plot, 
            'plot_log':plot_log, 
            'plot_legend':plot_legend,
            'plot_errorbars':True,
            'xticks':[datetime.datetime(2020, 3, 8), 
                      datetime.datetime(2020, 4, 15),  
                      datetime.datetime(2020, 5, 9)],                                                                             
            'x_range':[datetime.datetime(2020, 3, 8),
                   datetime.datetime(2020, 5, 9)],        
            'y_range': (0, 50000),
            'plot_daily_not_cumulative':plot_daily_not_cumulative,
            'model_line_label': 'Model predictions',
            'true_line_label': 'Reported cases',
            'title':title,
            'title_fontsize':20,
            'marker_size':5,
            'real_data_color':'tab:orange',
            'model_color':'tab:blue',
            'only_two_yticks':False,
            'mdl_prediction':mdl_predictions,
            'projected_hrs':projected_hrs}, 
        train_test_partition=train_test_partition,
        )    

    ax.grid(False)
    
log_scale_string = ', log scale' if plot_log else ''
daily_or_cumulative_string = 'daily' if plot_daily_not_cumulative else 'cumulative' 
title_string = ('Sorting by %s\nplotting %s %s%s' % (key_to_sort_by, 
                                                daily_or_cumulative_string, 
                                                thing_to_plot, 
                                                 log_scale_string))

ax = fig.add_subplot(111, frame_on=False)
ax.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_ylabel('Cumulative confirmed cases', fontsize=20)
ax.yaxis.set_label_coords(-0.11,0.5)

if prefix_to_save_plot_with is not None:
    for ext in ['svg']:
        plt.savefig('covid_figures_for_paper/trajectories/%s.%s' % 
                (prefix_to_save_plot_with,
                ext), 
                bbox_inches='tight',
                dpi=600)

plt.tight_layout()
plt.show()

In [ ]:
# Plot grid with top 20% models
thing_to_plot = 'cases'
plot_log = False
plot_daily_not_cumulative = False

for df, df_str in [(non_ablation_df, 'non_ablation')]:

    for train_test_partition, train_test_partition_str, key_to_sort_by in [
        (None, 'full_fit', 'loss_dict_daily_cases_RMSE')]:
        
        for thing_to_plot in ['cases']:
                        
            prefix_to_save_plot_with = f'trajectory_3x3_grid_{thing_to_plot}_{train_test_partition_str}_{df_str}_without_{msa_pretty_name}'
                    
            fig, axes = plt.subplots(3, 3, figsize=(11,9), sharex=True)
            axes_list = [ax for axes_row in axes for ax in axes_row]

            msas_to_plot_small = [msa for msa in MSAS if msa != msa_to_highlight]
            for ax, msa_name in zip(axes_list, msas_to_plot_small):

                subdf = df[(df['MSA_name'] == msa_name)].copy()# & (df['home_beta'] == 0.020)]
                subdf = subdf.sort_values(by=key_to_sort_by)
                losses = subdf[key_to_sort_by] / subdf.iloc[0][key_to_sort_by]
                num_models_to_aggregate = np.sum(losses <= ACCEPTABLE_LOSS_TOLERANCE)

                mdl_predictions = []
                old_projected_hrs = None
                for model_idx in range(num_models_to_aggregate):
                    ts = subdf.iloc[model_idx]['timestring']
                    _, kwargs, _, model_results, _ = load_model_and_data_from_timestring(ts, load_fast_results_only=False)
                    model_kwargs = kwargs['model_kwargs']
                    data_kwargs = kwargs['data_kwargs']
                    mdl_prediction, projected_hrs = plot_model_fit_from_model_and_kwargs(ax, 
                                                         model_kwargs, 
                                                         data_kwargs, 
                                                         model_results=model_results, 
                                                         plotting_kwargs={
                                                             'plot_mode':thing_to_plot, 
                                                             'make_plot':False,
                                                             'plot_daily_not_cumulative':plot_daily_not_cumulative,
                                                             'return_mdl_pred_and_hours':True},
                                                         train_test_partition=train_test_partition,
                                                     )
                    mdl_predictions.append(mdl_prediction)
                    if old_projected_hrs is not None:
                        assert projected_hrs == old_projected_hrs
                    old_projected_hrs = projected_hrs
                mdl_predictions = np.concatenate(mdl_predictions, axis=0)

                
                plot_model_fit_from_model_and_kwargs(
                    ax, 
                    model_kwargs, 
                    data_kwargs,         
                    plotting_kwargs={
                        'plot_mode':thing_to_plot, 
                        'plot_log':plot_log, 
                        'plot_legend':False,
                        'plot_errorbars':True,
                        'xticks':[datetime.datetime(2020, 3, 8),                                   
                                  datetime.datetime(2020, 5, 9)],                                                                             
                        'x_range':[datetime.datetime(2020, 3, 8),
                               datetime.datetime(2020, 5, 9)],                         
                        'plot_daily_not_cumulative':plot_daily_not_cumulative,                        
                        'title_fontsize':20,
                        'marker_size':5,
                        'real_data_color':'tab:orange',
                        'model_color':'tab:blue',
                        'only_two_yticks':True,
                        'mdl_prediction':mdl_predictions,
                        'projected_hrs':projected_hrs}, 
                    train_test_partition=train_test_partition,
                    )    


                ax.grid(False)
            log_scale_string = ', log scale' if plot_log else ''
            daily_or_cumulative_string = 'daily' if plot_daily_not_cumulative else 'cumulative' 
            title_string = ('Sorting by %s\nplotting %s %s%s' % (key_to_sort_by, 
                                                            daily_or_cumulative_string, 
                                                            thing_to_plot, 
                                                             log_scale_string))
            fig.subplots_adjust(wspace=0.3)

            if prefix_to_save_plot_with is not None:
                for ext in ['svg']:
                    plt.savefig('covid_figures_for_paper/trajectories/%s.%s' % 
                            (prefix_to_save_plot_with,
                            ext), 
                            dpi=600)

            print(prefix_to_save_plot_with)
            plt.show()

## Supplement, cases and deaths for all MSAs

In [ ]:
plot_log = False
plot_daily_not_cumulative = False

for df, df_str in [(non_ablation_df, 'non_ablation'), 
                   (ablation_df, 'ablation')]:

    for train_test_partition, train_test_partition_str, key_to_sort_by in [
        (None, 'full_fit', 'loss_dict_daily_cases_RMSE'), 
        (TRAIN_TEST_PARTITION, 'oos_fit', 'train_loss_dict_daily_cases_RMSE')]:
        
        for thing_to_plot in ['cases', 'deaths']:
                        
            prefix_to_save_plot_with = f'trajectory_5x2_grid_{thing_to_plot}_{train_test_partition_str}_{df_str}'
                    
            fig, axes = plt.subplots(5, 2, figsize=(8,15), sharex=True)
            axes_list = [ax for axes_row in axes for ax in axes_row]

            msas_to_plot_small = [msa for msa in MSAS]
            for ax_idx, (ax, msa_name) in enumerate(zip(axes_list, msas_to_plot_small)):

                subdf = df[(df['MSA_name'] == msa_name)].copy()
                subdf = subdf.sort_values(by=key_to_sort_by)
                losses = subdf[key_to_sort_by] / subdf.iloc[0][key_to_sort_by]
                num_models_to_aggregate = np.sum(losses <= ACCEPTABLE_LOSS_TOLERANCE)

                mdl_predictions = []
                old_projected_hrs = None
                for model_idx in range(num_models_to_aggregate):
                    ts = subdf.iloc[model_idx]['timestring']
                    _, kwargs, _, model_results, _ = load_model_and_data_from_timestring(ts, load_fast_results_only=False)
                    model_kwargs = kwargs['model_kwargs']
                    data_kwargs = kwargs['data_kwargs']
                    mdl_prediction, projected_hrs = plot_model_fit_from_model_and_kwargs(ax, 
                                                         model_kwargs, 
                                                         data_kwargs, 
                                                         model_results=model_results, 
                                                         plotting_kwargs={
                                                             'plot_mode':thing_to_plot, 
                                                             'make_plot':False,
                                                             'plot_daily_not_cumulative':plot_daily_not_cumulative,
                                                             'return_mdl_pred_and_hours':True},
                                                         train_test_partition=train_test_partition,
                                                     )
                    mdl_predictions.append(mdl_prediction)
                    if old_projected_hrs is not None:
                        assert projected_hrs == old_projected_hrs
                    old_projected_hrs = projected_hrs
                mdl_predictions = np.concatenate(mdl_predictions, axis=0)
                
                if thing_to_plot == 'cases':
                    x_start = datetime.datetime(2020, 3, 8)
                    real_data_color = 'tab:orange'
                    model_color = 'tab:blue'
                    model_line_label = 'Model predictions'
                    true_line_label = 'Reported cases'                 
                elif thing_to_plot == 'deaths':
                    x_start = datetime.datetime(2020, 3, 19)
                    real_data_color = 'saddlebrown'
                    model_color = 'mediumblue'
                    model_line_label = 'Model predictions'
                    true_line_label = 'Reported deaths'
                                    
                if train_test_partition_str == 'oos_fit':
                    xticks = [x_start, 
                              datetime.datetime(2020, 4, 15),
                              datetime.datetime(2020, 5, 9)]
                else:
                    xticks = [x_start,                               
                              datetime.datetime(2020, 5, 9)]                    
                    
                plot_model_fit_from_model_and_kwargs(
                    ax, 
                    model_kwargs, 
                    data_kwargs,         
                    plotting_kwargs={
                        'plot_mode':thing_to_plot, 
                        'plot_log':plot_log, 
                        'plot_legend':False,
                        'plot_errorbars':True,
                        'xticks':xticks,
                        'x_range':[x_start,
                                   datetime.datetime(2020, 5, 9)],
                        'plot_daily_not_cumulative':plot_daily_not_cumulative,
                        'title_fontsize':20,
                        'marker_size':5,
                        'model_line_label': model_line_label,
                        'true_line_label': true_line_label,
                        'real_data_color':real_data_color,
                        'model_color':model_color,
                        'only_two_yticks':True,
                        'mdl_prediction':mdl_predictions,
                        'projected_hrs':projected_hrs}, 
                    train_test_partition=train_test_partition,
                    )    
                if ax_idx == 0:
                    ax.legend(bbox_to_anchor=(-0.4, 1.04))

                ax.grid(False)                                
                
            log_scale_string = ', log scale' if plot_log else ''
            daily_or_cumulative_string = 'daily' if plot_daily_not_cumulative else 'cumulative' 
            title_string = ('Sorting by %s\nplotting %s %s%s' % (key_to_sort_by, 
                                                            daily_or_cumulative_string, 
                                                            thing_to_plot, 
                                                             log_scale_string))
            fig.subplots_adjust(wspace=0.3)
            

            if prefix_to_save_plot_with is not None:
                for ext in ['svg']:
                    plt.savefig('covid_figures_for_paper/trajectories/%s.%s' % 
                            (prefix_to_save_plot_with,
                            ext), 
                            bbox_inches='tight',
                            dpi=600)

            print(prefix_to_save_plot_with)
            plt.show()                                        

# Mobility reductions + reopening analysis

## Figure 2a: counterfactual mobility reduction

### Figure 2a, left: schematic

In [ ]:
def apply_distancing_degree(poi_cbg_visits_list, distancing_degree):
    new_visits_list = []
    for i, m in enumerate(poi_cbg_visits_list):
        if i < 168:  # first week
            new_visits_list.append(m.copy())
        else:
            first_week_m = poi_cbg_visits_list[i % 168]
            mixture = first_week_m.multiply(1-distancing_degree) + m.multiply(distancing_degree)
            new_visits_list.append(mixture.copy())
    return new_visits_list

def apply_shift_in_days(poi_cbg_visits_list, shift_in_days):
    new_visits_list = []
    shift_in_hours = shift_in_days * 24
    if shift_in_hours <= 0:  # shift earlier
        new_visits_list = [m.copy() for m in poi_cbg_visits_list[abs(shift_in_hours):]]
        current_length = len(new_visits_list)
        assert current_length >= 168
        last_week = new_visits_list[-168:]
        for i in range(current_length, len(poi_cbg_visits_list)):
            last_week_counterpart = last_week[i % 168].copy()
            new_visits_list.append(last_week_counterpart)
    else:  # shift later
        for i in range(len(poi_cbg_visits_list)):
            if i-shift_in_hours < 0:
                distance_from_start = (shift_in_hours - i) % 168
                first_week_idx = 168 - distance_from_start
                new_visits_list.append(poi_cbg_visits_list[first_week_idx].copy())
            else:
                new_visits_list.append(poi_cbg_visits_list[i-shift_in_hours].copy())
    return new_visits_list

In [ ]:
def get_daily_ts(poi_cbg_visits_list):
    daily_ts = []
    for i in np.arange(0, len(poi_cbg_visits_list), 24):
        day_total = 0
        for j in range(i, i+24):
            day_total += poi_cbg_visits_list[j].sum()
        daily_ts.append(day_total)
    return daily_ts

# https://dfrieds.com/data-visualizations/how-format-large-tick-values.html
def reformat_large_tick_values(tick_val, pos):
    """
    Turns large tick values (in the billions, millions and thousands) such as 4500 into 4.5K and also appropriately turns 4000 into 4K (no zero after the decimal).
    """
    if tick_val >= 1000000000:
        val = round(tick_val/1000000000, 1)
        new_tick_format = '{:}B'.format(val)
    elif tick_val >= 1000000:
        val = round(tick_val/1000000, 1)
        new_tick_format = '{:}M'.format(val)
    elif tick_val >= 1000:
        val = round(tick_val/1000, 1)
        new_tick_format = '{:}K'.format(val)
    elif tick_val < 1000:
        new_tick_format = round(tick_val, 1)
    else:
        new_tick_format = tick_val

    # make new_tick_format into a string value
    new_tick_format = str(new_tick_format)
                
    return new_tick_format
    
def make_schematic(orig_visits, lesser_extent, shifted, colors, ax):
    days = helper.list_datetimes_in_range(MIN_DATETIME, MAX_DATETIME)
    daily_ts = get_daily_ts(orig_visits)
    ax.plot_date(days, daily_ts, linestyle='-', marker='.', color=colors[0], label='actual',
                linewidth=3)
    daily_ts = get_daily_ts(lesser_extent)
    ax.plot_date(days, daily_ts, linestyle='-', marker='.', color=colors[1], label='50% of actual',
                linewidth=3, alpha=0.5)
    daily_ts = get_daily_ts(shifted)
    ax.plot_date(days, daily_ts, linestyle='-', marker='.', color=colors[2], label='7 days later',
                linewidth=3, alpha=0.5)
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(mdates.SU, interval=2))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    ax.yaxis.set_major_formatter(tick.FuncFormatter(reformat_large_tick_values))
    ax.legend(fontsize=16)
    ax.grid(alpha=0.3)
    ax.set_ylabel('Total POI visits per day', fontsize=16)
    ax.tick_params(labelsize=16)
    ax.set_xlabel('Date', fontsize=16)

In [ ]:
lesser_extent = apply_distancing_degree(orig, 0.5)
shifted = apply_shift_in_days(orig, 7)
colors = ['black', degree_colors[2], shift_colors[0]]
fig, ax = plt.subplots(figsize=(7, 7))
make_schematic(orig, lesser_extent, shifted, colors, ax)

### Figure 2a, middle and right: results

In [ ]:
# load models
counterfactual_results = []
min_timestring = '2020_05_29_15_07_37_648695'
max_timestring = '2020_06_16'
required_properties = {'experiment_to_run':'test_retrospective_counterfactuals'}
for msa in MSAS:
    counterfactual_results.append(evaluate_all_fitted_models_for_msa(msa, 
                                                          min_timestring=min_timestring,
                                                          max_timestring=max_timestring,
                                                          required_properties=required_properties,
                                                          key_to_sort_by=None))
counterfactual_df = pd.concat(counterfactual_results)
counterfactual_df['MSA_name'] = counterfactual_df['data_kwargs'].map(lambda x:x['MSA_name'])
assert (counterfactual_df['poi_psi'] > 0).all()
print("Total models loaded after filtering for counterfactual experiments: %i" % len(counterfactual_df))

In [ ]:
for k in ['distancing_degree', 'shift_in_days']:
    counterfactual_df['counterfactual_%s' % k] = counterfactual_df['counterfactual_retrospective_experiment_kwargs'].map(lambda x:x[k] if k in x else np.nan)
counterfactual_df['counterfactual_baseline_model'] = counterfactual_df['counterfactual_retrospective_experiment_kwargs'].map(lambda x:x['model_quality_dict']['model_timestring'])
counterfactual_df.groupby('MSA_name').size()

In [ ]:
def plot_lir_over_time_for_multiple_models(timestrings, ax, label, color):
    all_lir = []
    hours = None
    for ts in timestrings:
        _, kwargs, _, model_results, _ = load_model_and_data_from_timestring(ts, load_fast_results_only=False,
                                                                             load_full_model=False)

        new_hours = [kwargs['model_kwargs']['min_datetime'] + datetime.timedelta(hours=a)
                 for a in range(model_results['history']['all']['latent'].shape[1])]
        if hours is None:
            hours = new_hours
        else:
            assert list(hours) == list(new_hours) # make sure hours stays unchanged between timestrings.
        lir = (model_results['history']['all']['latent'] +
           model_results['history']['all']['infected'] +
           model_results['history']['all']['removed']) / model_results['history']['all']['total_pop']
        all_lir.append(lir)
    all_lir = np.concatenate(all_lir, axis=0)
    print('Num params x seeds:', all_lir.shape[0])
    mean = INCIDENCE_POP * np.mean(all_lir, axis=0)
    lower_bound = INCIDENCE_POP * np.percentile(all_lir, LOWER_PERCENTILE, axis=0)
    upper_bound = INCIDENCE_POP * np.percentile(all_lir, UPPER_PERCENTILE, axis=0)
    ax.plot_date(hours, mean, linestyle='-', label=label, color=color)
    ax.fill_between(hours, lower_bound, upper_bound, alpha=0.5, color=color)
    ax.set_xlim([min(hours), max(hours)])

def make_counterfactual_line_plots(counterfactual_df, msa, ax, mode,
                                   cmap_str='viridis', y_lim=None):
    assert mode in {'degree', 'shift-later', 'shift-earlier', 'shift'}

    
    if mode == 'degree':
        colors = list(cm.get_cmap(cmap_str, 5).colors)
        colors.reverse()
        values = [0, .25, .5, np.nan]  # put highest curve first, so that legend order is correct
        param_name = 'counterfactual_distancing_degree'
        subtitle = 'What if we reduced social distancing?'
    else:
        colors = list(cm.get_cmap(cmap_str, 6).colors)
        colors.reverse()
        colors = colors[1:]
        if mode == 'shift-later':
            values = [14, 7, 3, np.nan]
            param_name = 'counterfactual_shift_in_days'
            subtitle = 'What if we had started socially distancing x days later?'
        elif mode == 'shift-earlier':
            values = [np.nan, -3, -7, -14]
            param_name = 'counterfactual_shift_in_days'
            subtitle = 'What if we had started socially distancing x days earlier?'
            colors = colors[3:]  # so that true curve maintains the same color
        else:
            values = [7, 3, np.nan, -3, -7]
            param_name = 'counterfactual_shift_in_days'
            subtitle = 'What if we had started earlier or later?'

    msa_df = counterfactual_df[counterfactual_df['MSA_name'] == msa]
    color_idx = 0
    for i, val in enumerate(values):
        if np.isnan(val):
            # plot baseline models for comparison
            timestrings = msa_df.counterfactual_baseline_model.unique()
            label = 'actual' if mode == 'degree' else '0 days (actual)'
            plot_lir_over_time_for_multiple_models(timestrings, ax, label, 'black')
        else:
            # plot the models from this experiment
            msa_val_df = msa_df[msa_df[param_name] == val]
            timestrings = msa_val_df.timestring.values
            if mode == 'degree':
                label = 'no reduction' if val == 0 else '%d%% of actual' % (val * 100)
            else:  # mode is some kind of shift
                postfix = 'earlier' if val < 0 else 'later'
                label = '%d days %s' % (abs(val), postfix)  # take abs value in case val is negative
            plot_lir_over_time_for_multiple_models(timestrings, ax, label, colors[i])

    ax.legend(loc='upper left', fontsize=16)
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.SU, interval=2))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    ax.yaxis.set_major_formatter(tick.FuncFormatter(reformat_with_k))
    ax.set_xlabel('Date', fontsize=18)
    if y_lim is not None:
        ax.set_ylim(y_lim)
    ax.grid(alpha=.5)
    ax.tick_params(labelsize=16)
    ax.set_title(subtitle, fontsize=18)
    return colors 

In [ ]:
def plot_lir_over_time_for_multiple_models(timestrings, ax, label, color):
    all_lir = []
    hours = None
    for ts in timestrings:
        _, kwargs, _, model_results, _ = load_model_and_data_from_timestring(ts, load_fast_results_only=False,
                                                                             load_full_model=False)

        new_hours = [kwargs['model_kwargs']['min_datetime'] + datetime.timedelta(hours=a)
                 for a in range(model_results['history']['all']['latent'].shape[1])]
        if hours is None:
            hours = new_hours
        else:
            assert list(hours) == list(new_hours) # make sure hours stays unchanged between timestrings.
        lir = (model_results['history']['all']['latent'] +
           model_results['history']['all']['infected'] +
           model_results['history']['all']['removed']) / model_results['history']['all']['total_pop']
        all_lir.append(lir)
    all_lir = np.concatenate(all_lir, axis=0)
    print('Num params x seeds:', all_lir.shape[0])
    mean = np.mean(all_lir, axis=0)
    lower_bound = np.percentile(all_lir, LOWER_PERCENTILE, axis=0)
    upper_bound = np.percentile(all_lir, UPPER_PERCENTILE, axis=0)
    ax.plot_date(hours, mean, linestyle='-', label=label, color=color)
    ax.fill_between(hours, lower_bound, upper_bound, alpha=0.5, color=color)
    ax.set_xlim([min(hours), max(hours)])

def make_counterfactual_line_plots(counterfactual_df, msa, ax, mode,
                                   cmap_str='viridis', y_lim=None):
    assert mode in {'degree', 'shift-later', 'shift-earlier', 'shift'}

    colors = list(cm.get_cmap(cmap_str, 5).colors)
    colors.reverse()
    if mode == 'degree':
        values = [0, .25, .5, np.nan]  # put highest curve first, so that legend order is correct
        param_name = 'counterfactual_distancing_degree'
        subtitle = 'What if we reduced social distancing?'
        colors = colors[-len(values):]
    elif mode == 'shift-later':
        values = [14, 7, 3, np.nan]
        param_name = 'counterfactual_shift_in_days'
        subtitle = 'What if we had started socially distancing x days later?'
    elif mode == 'shift-earlier':
        values = [np.nan, -3, -7, -14]
        param_name = 'counterfactual_shift_in_days'
        subtitle = 'What if we had started socially distancing x days earlier?'
        colors = colors[3:]  # so that true curve maintains the same color
    else:
        values = [7, 3, np.nan, -3, -7]
        param_name = 'counterfactual_shift_in_days'
        subtitle = 'What if we had started earlier or later?'

    msa_df = counterfactual_df[counterfactual_df['MSA_name'] == msa]
    color_idx = 0
    for i, val in enumerate(values):
        if np.isnan(val):
            # plot baseline models for comparison
            timestrings = msa_df.counterfactual_baseline_model.unique()
            label = 'actual' if mode == 'degree' else '0 days (actual)'
            plot_lir_over_time_for_multiple_models(timestrings, ax, label, colors[i])
        else:
            # plot the models from this experiment
            msa_val_df = msa_df[msa_df[param_name] == val]
            timestrings = msa_val_df.timestring.values
            if mode == 'degree':
                label = '%d%%' % (val * 100)
            else:  # mode is some kind of shift
                postfix = 'earlier' if val < 0 else 'later'
                label = '%d days %s' % (abs(val), postfix)  # take abs value in case val is negative
            plot_lir_over_time_for_multiple_models(timestrings, ax, label, colors[i])

    ax.legend(loc='upper left', fontsize=16)
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.SU, interval=2))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    ax.set_xlabel('Date', fontsize=18)
    ax.set_ylabel('Fraction of population infected', fontsize=18)
    if y_lim is not None:
        ax.set_ylim(y_lim)
    ax.grid(alpha=.5)
    ax.tick_params(labelsize=16)
    ax.set_title(subtitle, fontsize=18)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 7))
y_lim = (0, 27000)
degree_colors = make_counterfactual_line_plots(counterfactual_df, DC_NAME, axes[0], 'degree', y_lim=y_lim)
axes[0].set_title('')
shift_colors = make_counterfactual_line_plots(counterfactual_df, DC_NAME, axes[1], 'shift', cmap_str='plasma', y_lim=y_lim)
axes[1].set_title('')
axes[1].set_ylabel('')
plt.show()

## Supplement: table of counterfactual to real infection outcome ratios

In [ ]:
def get_final_LIR_fraction_for_multiple_models(timestrings):
    final_frac = []
    for ts in timestrings:
        _, kwargs, _, model_results, _ = load_model_and_data_from_timestring(ts, 
                                                                             load_fast_results_only=False, 
                                                                             load_full_model=False)
        frac_in_lir = ((model_results['history']['all']['latent'] + 
                       model_results['history']['all']['infected'] + 
                       model_results['history']['all']['removed']) /
                       model_results['history']['all']['total_pop'])
        final_frac.extend(frac_in_lir[:, -1])
    return np.array(final_frac)

def get_ratios_to_baselines(msa_df, param_name, param_values, baselines):
    cols = []
    all_ratios = []
    for i, val in enumerate(param_values):
        msa_val_df = msa_df[msa_df[param_name] == val]
        timestrings = msa_val_df.timestring.values
        final_fracs = get_final_LIR_fraction_for_multiple_models(timestrings)
        ratios = final_fracs / baselines
        cols.append('%s=%s' % (param_name.strip('counterfactual_'), val))
        mean = np.mean(ratios)
        lower_bound = np.percentile(ratios, LOWER_PERCENTILE)
        upper_bound = np.percentile(ratios, UPPER_PERCENTILE)
        all_ratios.append((round(mean, 3), (round(lower_bound, 3), round(upper_bound, 3))))
    return cols, all_ratios
    
def get_counterfactual_ratios_at_datetime(counterfactual_df, msa):
    msa_df = counterfactual_df[counterfactual_df['MSA_name'] == msa].copy()
    msa_df = msa_df.sort_values(by='counterfactual_baseline_model')
    baseline_timestrings = sorted(msa_df.counterfactual_baseline_model.unique())
    baselines = get_final_LIR_fraction_for_multiple_models(baseline_timestrings)
    
    param_name = 'counterfactual_distancing_degree'
    param_values = [0, .25, .5]    
    cols, ratios = get_ratios_to_baselines(msa_df, param_name, param_values, baselines)
    param_name = 'counterfactual_shift_in_days'
    param_values = [7, 3, -3, -7]
    cols2, ratios2 = get_ratios_to_baselines(msa_df, param_name, param_values, baselines)
    cols.extend(cols2)
    ratios.extend(ratios2)
    return cols, ratios

In [ ]:
all_ratios = []
cols = None
for msa in MSAS:
    print(msa)
    cols, ratios = get_counterfactual_ratios_at_datetime(counterfactual_df, msa)
    ratios.insert(0, MSAS_TO_PRETTY_NAMES[msa])
    all_ratios.append(ratios)
cols.insert(0, 'MSA_name')
ratios_df = pd.DataFrame(all_ratios, columns=cols)
ratios_df

## Figure 2b: super-spreader POIs

In [ ]:
superspreader_poi_df = []
min_timestring = '2020_05_30_14_28_28_220347'
required_properties = {'experiment_to_run':'rerun_best_models_and_save_cases_per_poi'}
for msa in msas:
    superspreader_poi_df.append(evaluate_all_fitted_models_for_msa(msa, 
                                                      min_timestring=min_timestring,
                                                      required_properties=required_properties,
                                                      key_to_sort_by=None))
superspreader_poi_df = pd.concat(superspreader_poi_df)
superspreader_poi_df['MSA_name'] = superspreader_poi_df['data_kwargs'].map(lambda x:x['MSA_name'])
print("Total models loaded after filtering for superspreader POI analysis: %i" % len(superspreader_poi_df))

In [ ]:
poi_characteristics = pickle.load(open('poi_characteristics.pkl', 'rb'))
min_datestring = None
max_datestring = None
fig_with_all_pois = plt.figure(1, figsize=[10, 20])
msa_idx = 0
all_proportions_of_total_infections_from_pois = []

for msa in msas:
    print(msa)
    city_df = superspreader_poi_df.loc[superspreader_poi_df['MSA_name'] == msa]
    all_poi_counts_for_city = []
    city_proportions_of_total_infections_from_pois = []
    for i in range(len(city_df)):
        timestring = city_df.iloc[i]['timestring']
        mdl, kwargs, _, _, _ = load_model_and_data_from_timestring(
                timestring,
                load_fast_results_only=False, 
                load_full_model=True)
        city_proportions_of_total_infections_from_pois += list((mdl.history['all']['new_cases_from_poi'].sum(axis=1)/
                                mdl.history['all']['new_cases'].sum(axis=1)))
        if min_datestring is None:
            min_datestring = kwargs['model_kwargs']['min_datetime'].strftime('%B %-d')
        else:
            assert min_datestring == kwargs['model_kwargs']['min_datetime'].strftime('%B %-d')
            
        if max_datestring is None:
            max_datestring = kwargs['model_kwargs']['max_datetime'].strftime('%B %-d')
        else:
            assert max_datestring == kwargs['model_kwargs']['max_datetime'].strftime('%B %-d')
            
        all_poi_counts_for_city.append(mdl.history['all']['num_cases_per_poi'])
        if len(all_poi_counts_for_city) > 1:
            assert all_poi_counts_for_city[-1].shape == all_poi_counts_for_city[-2].shape
            
    all_proportions_of_total_infections_from_pois.append(pd.DataFrame(
        {'msa':msa,'prop_total_infections_from_pois':city_proportions_of_total_infections_from_pois}))
    # all_poi_counts_for_city is n_seeds x n_pois
    all_poi_counts_for_city = np.concatenate(all_poi_counts_for_city, axis=0)
    all_poi_fracs_for_city = all_poi_counts_for_city/all_poi_counts_for_city.sum(axis=1).reshape([len(all_poi_counts_for_city), 1])
    assert np.allclose(all_poi_fracs_for_city.sum(axis=1), 1)
    
    
    
    mean_frac_per_poi = np.mean(all_poi_fracs_for_city, axis=0)
    poi_characteristics_df = poi_characteristics[msa].copy()
    poi_characteristics_df['mean_frac_of_infections_at_poi'] = mean_frac_per_poi
    poi_characteristics_df['density*dwell_time_factor'] = poi_characteristics_df['dwell_time_correction_factors'] * poi_characteristics_df['weighted_visits_over_area']
    poi_characteristics_df['visits^2*dwell_time_factor/area'] = poi_characteristics_df['dwell_time_correction_factors'] * poi_characteristics_df['squared_visits_over_area']
    poi_characteristics_df['weighted_visits'] = poi_characteristics_df['weighted_visits_over_area'] * poi_characteristics_df['poi_areas']
    poi_characteristics_df['dwell_time'] = invert_dwell_time_correction_factors(poi_characteristics_df['dwell_time_correction_factors'].values)
    cutoff_90 = scoreatpercentile(poi_characteristics_df['mean_frac_of_infections_at_poi'], 90)
    cutoff_99 = scoreatpercentile(poi_characteristics_df['mean_frac_of_infections_at_poi'], 99)
    poi_characteristics_df['infectiousness_group'] = poi_characteristics_df['mean_frac_of_infections_at_poi'].map(lambda x:'top 10%' if x >= cutoff_90
                                                                                                                  else 'bottom 90%')
    
    print("Spearman correlations (across POIs) between POI characteristics and fraction of total infections at POI")
    print(poi_characteristics_df.corr(method='spearman')['mean_frac_of_infections_at_poi'])
    print(poi_characteristics_df.groupby('infectiousness_group').median().transpose()[['bottom 90%', 'top 10%']])
    
    sorted_idxs = np.argsort(mean_frac_per_poi)[::-1]
    all_poi_fracs_for_city = all_poi_fracs_for_city[:, sorted_idxs]
    cumulative_poi_fracs = np.cumsum(all_poi_fracs_for_city, axis=1)
    x = np.linspace(0, 1, cumulative_poi_fracs.shape[1])
    alpha = 0.05
    lower_CI = np.percentile(cumulative_poi_fracs, 100 * alpha/2, axis=0)
    upper_CI = np.percentile(cumulative_poi_fracs, 100 * (1 - alpha/2), axis=0)
    
    
    fig = plt.figure(figsize=[6, 6])
    for subplot_idx, plot_log in enumerate([False]):
        ax = fig.add_subplot(1, 1, subplot_idx + 1)

        ax.plot(x, cumulative_poi_fracs.mean(axis=0), color='tab:blue')
        ax.fill_between(x, lower_CI, upper_CI, color='tab:blue', alpha=.2)
        ax.set_xlim([1e-4, 1])
        ax.set_ylim([1e-4, 1])
        ax.set_xlabel("Percent of POIs", fontsize=14)
        ax.set_ylabel("Percent of POI infections", fontsize=14)
        if plot_log:
            ax.set_xscale('log')
            ax.set_yscale('log')
        else:
            plt.xticks(np.arange(0.0, 1.01, .1), 
                          ['0%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
            plt.yticks(np.arange(0.0, 1.01, .1), 
                          ['0%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
        ax.grid(alpha=.3)
        ax.set_title('Distribution of infections over POIs (%s-%s)' % (min_datestring, max_datestring), fontsize=14)
    fig.savefig('covid_figures_for_paper/superspreader_poi_cdf_%s.pdf' % msa, bbox_inches='tight')
    
    # also make figure for all POIs. 
    plt.figure(1) # make sure we're modifying the right figure
    ax = fig_with_all_pois.add_subplot(5, 2, msa_idx + 1)
    ax.plot(x, cumulative_poi_fracs.mean(axis=0), color='tab:blue')
    ax.fill_between(x, lower_CI, upper_CI, color='tab:blue', alpha=.2)
    ax.set_xlim([1e-4, 1])
    ax.set_ylim([1e-4, 1])
    if msa_idx % 2 == 0:
        ax.set_ylabel("Percent of POI infections", fontsize=14)
        
    if msa_idx >= 8:
        ax.set_xlabel("Percent of POIs", fontsize=14)
    plt.xticks(np.arange(0.0, 1.01, .1), 
                    ['0%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
    plt.yticks(np.arange(0.0, 1.01, .1), 
                  ['0%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
    ax.grid(alpha=.3)
    ax.set_title(MSAS_TO_PRETTY_NAMES[msa])
    msa_idx += 1
    
#fig_with_all_pois.suptitle('Distribution of infections over POIs (%s-%s)' % (min_datestring, max_datestring), fontsize=14)
fig_with_all_pois.savefig('covid_figures_for_paper/superspreader_poi_cdf_all_msas.png', bbox_inches='tight', dpi=150)
plt.show()

## Figure 2c: partial reopening strategies

In [ ]:
# load models
max_cap_results = []
min_timestring = '2020_05_29_09_48_56_478371'
max_timestring = '2020_06_16'
required_properties = {'experiment_to_run':'test_max_capacity_clipping'}
for msa in MSAS:
    max_cap_results.append(evaluate_all_fitted_models_for_msa(msa, 
                                                          min_timestring=min_timestring,
                                                          max_timestring=max_timestring,
                                                          required_properties=required_properties,
                                                          key_to_sort_by=None))
max_cap_df = pd.concat(max_cap_results)
max_cap_df['MSA_name'] = max_cap_df['data_kwargs'].map(lambda x:x['MSA_name'])
assert (max_cap_df['poi_psi'] > 0).all()
print("Total models loaded after filtering for max capacity experiments: %i" % len(max_cap_df))

In [ ]:
k = 'max_capacity_alpha'
max_cap_df['counterfactual_%s' % k] = max_cap_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x[k] if k in x else np.nan)
max_cap_df['counterfactual_baseline_model'] = max_cap_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x['model_quality_dict']['model_timestring'])
max_cap_df['baseline_model_quality'] = max_cap_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x['model_quality_dict']['model_fit_rank_for_msa'])
max_cap_df.groupby('MSA_name').size()

In [ ]:
uniform_results = []
min_timestring = '2020_05_31_20_38_59_228416'
required_properties = {'experiment_to_run':'test_uniform_proportion_of_full_reopening'}
for msa in MSAS:
    uniform_results.append(evaluate_all_fitted_models_for_msa(msa, 
                                                      min_timestring=min_timestring,
                                                      max_timestring=max_timestring,
                                                      required_properties=required_properties,
                                                      key_to_sort_by=None))
uniform_df = pd.concat(uniform_results)
uniform_df['MSA_name'] = uniform_df['data_kwargs'].map(lambda x:x['MSA_name'])
assert (uniform_df['poi_psi'] > 0).all()
print("Total models loaded after filtering for uniform reopening experiments: %i" % len(uniform_df))

In [ ]:
k = 'full_activity_alpha'
uniform_df['counterfactual_%s' % k] = uniform_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x[k] if k in x else np.nan)
uniform_df['counterfactual_baseline_model'] = uniform_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x['model_quality_dict']['model_timestring'])
uniform_df['baseline_model_quality'] = uniform_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x['model_quality_dict']['model_fit_rank_for_msa'])
uniform_df.groupby('MSA_name').size()

In [ ]:
def get_full_activity_num_visits(msa, intervention_datetime, extra_weeks_to_simulate, min_datetime, max_datetime):
    fn = get_ipf_filename(msa, min_datetime, max_datetime, True, True)
    f = open(fn, 'rb')
    poi_cbg_visits_list = pickle.load(f)
    f.close()
    all_hours = helper.list_hours_in_range(min_datetime, max_datetime + datetime.timedelta(hours=168 * extra_weeks_to_simulate))
    assert(intervention_datetime in all_hours)
    intervention_hour_idx = all_hours.index(intervention_datetime)
    full_total = 0
    for t in range(intervention_hour_idx, len(all_hours)):
        full_activity_matrix = poi_cbg_visits_list[t % 168]
        full_total += full_activity_matrix.sum()
    return full_total, intervention_hour_idx

### Figure 2c, left: impacts of clipping strategy

In [ ]:
def reformat_with_k(tick_val, pos):
    new_tick_val = int(round(tick_val / 1000, 0))
    # make new_tick_format into a string value
    new_tick_format = '%dk' % new_tick_val                
    return new_tick_format

def get_lir_checkpoints_and_prop_visits_lost(timestring, intervention_hour_idx, 
                                             normalize_lir=True, full_activity_num_visits=None,
                                             group='all'):
    _, _, _, model_results, fast_to_load_results = load_model_and_data_from_timestring(timestring, 
                                                                                       load_fast_results_only=False,
                                                                                       load_full_model=False)
    lir = (model_results['history'][group]['latent'] +
           model_results['history'][group]['infected'] +
           model_results['history'][group]['removed'])
    intervention_lir = lir[:, intervention_hour_idx]
    final_lir = lir[:, -1]
    if normalize_lir:
        intervention_lir = intervention_lir / model_results['history'][group]['total_pop']
        final_lir = final_lir / model_results['history'][group]['total_pop']
    intervention_cost = fast_to_load_results['intervention_cost']
    if 'total_activity_after_max_capacity_capping' in intervention_cost:
        assert full_activity_num_visits is not None
        num_visits = intervention_cost['total_activity_after_max_capacity_capping']
        visits_lost = (full_activity_num_visits - num_visits) / full_activity_num_visits
    else:
        assert 'overall_cost' in intervention_cost
        visits_lost = intervention_cost['overall_cost'] / 100
    return intervention_lir, final_lir, visits_lost
    
def make_pareto_plot(results_df, msa, param_name, intervention_idx,
                     ax, color=None, cbg_group='all', annotation_color=None, annotate_points=True,
                     full_activity_num_visits=None, line_label=None, 
                     set_axis_labels=True, add_X=None, add_Y=None, add_label=None,
                     plot_intervention_lir_line=False, plot_lir_diff=False, only_best_model=False):    
    assert not(plot_intervention_lir_line and plot_lir_diff)
    msa_df = results_df[results_df['MSA_name'] == msa]
    values = sorted([a for a in msa_df[param_name].unique()])
    X = []  # prop visits lost
    Y_min = []  # frac infections, min
    Y_mean = []  # frac infections, mean
    Y_max = []  # frac infections, max
    all_intervention_lir = []  # frac infections, at the point of intervention
    for i, val in enumerate(values):
        msa_val_df = msa_df[msa_df[param_name] == val]
        if only_best_model:
            msa_val_df = msa_val_df[msa_val_df['baseline_model_quality'] == 1]
        timestrings = msa_val_df.timestring.values
        visits_lost = None
        final_lir = []
        for ts in timestrings:
            curr_intervention_lir, curr_final_lir, curr_visits_lost = get_lir_checkpoints_and_prop_visits_lost(ts, intervention_idx, 
                                                                                                               group=cbg_group, full_activity_num_visits=full_activity_num_visits)
            all_intervention_lir.extend(list(curr_intervention_lir))
            if plot_lir_diff:
                final_lir.extend(list(curr_final_lir - curr_intervention_lir))
            else:
                final_lir.extend(list(curr_final_lir))
            if visits_lost is None:
                visits_lost = curr_visits_lost
            else:
                assert visits_lost == curr_visits_lost
        X.append(visits_lost)
        Y_min.append(INCIDENCE_POP * np.percentile(final_lir, LOWER_PERCENTILE))
        Y_mean.append(INCIDENCE_POP * np.mean(final_lir))
        Y_max.append(INCIDENCE_POP * np.percentile(final_lir, UPPER_PERCENTILE))
        if i == 0:
            print('Num params * seeds:', len(final_lir))
    if add_X is not None and add_Y is not None:
        X.append(add_X)
        Y_min.append(add_Y[0])
        Y_mean.append(add_Y[1])
        Y_max.append(add_Y[2])
        values.append(1.0)
    
    if ax is None:
        return X, Y_min, Y_mean, Y_max
    
    if line_label:
        ax.plot(X, Y_mean, marker='o', linewidth=2, color=color, label=line_label)
    else:
        ax.plot(X, Y_mean, marker='o', linewidth=2, color=color)
    ax.fill_between(X, Y_min, Y_max, alpha=0.3, color=color)
    if annotate_points:
        assert annotation_color is not None
        for val, x, y in zip(values, X, Y_mean):
            if val < .8:
                ax.annotate('%d%%' % (100 * val), xy=(x, y), xytext=(0, 5), textcoords='offset pixels', fontsize=16, color=annotation_color)
    if plot_intervention_lir_line:
        intervention_lir_mean = np.mean(all_intervention_lir)
        ax.plot([min(X), max(X)], [INCIDENCE_POP * intervention_lir_mean, INCIDENCE_POP * intervention_lir_mean], color='red', linestyle='--', label='cumulative infections\nbefore reopening')
    ax.yaxis.set_major_formatter(tick.FuncFormatter(reformat_with_k))
    if set_axis_labels:
        ax.set_xlabel('Fraction of visits lost in month after reopening\n(compared to full reopening)', fontsize=16)
        if plot_lir_diff:
            ax.set_ylabel('New infections (per 100k)\nin month after reopening', fontsize=16)
        else:
            ax.set_ylabel('Cumulative infections (per 100k)', fontsize=16)
        ax.legend(fontsize=16)
    ax.grid(alpha=.3)
    ax.tick_params(labelsize=16)
    return X, Y_min, Y_mean, Y_max

In [ ]:
dc_full_activity, intervention_idx = get_full_activity_num_visits(DC_NAME, datetime.datetime(2020, 5, 1, 0),
                                                                  4, MIN_DATETIME, MAX_DATETIME)
print(dc_full_activity, intervention_idx)
X, Y_min, Y_mean, Y_max = make_pareto_plot(uniform_df, DC_NAME, 'counterfactual_full_activity_alpha', 
                                           intervention_idx, None)
assert math.isclose(X[-1], 0, abs_tol=1e-8)
fig, ax = plt.subplots(figsize=(7, 7))
X, Y_min, Y_mean, Y_max = make_pareto_plot(max_cap_df, DC_NAME, 'counterfactual_max_capacity_alpha', intervention_idx,
                 ax, 'tab:blue', annotation_color='black',
                 full_activity_num_visits=dc_full_activity,
                 set_axis_labels=False, add_X=X[-1], add_Y=(Y_min[-1], Y_mean[-1], Y_max[-1]), plot_intervention_lir_line=True)
print('Visits lost:', X)
print('Visits kept:', 1 - np.array(X))
print('Infections:', Y_mean)
x_lim = ax.get_xlim()
plt.show()

### Figure 2c, right: comparison of clipping and uniform reduction strategies

In [ ]:
def reformat_decimal_as_percent(tick_val, pos):
    percent = round(tick_val * 100, 1)
    new_tick_format = '%d%%' % percent
    return new_tick_format

def plot_pairwise_comparison(max_cap_df, uniform_df, msa_name, full_activity_num_visits, intervention_idx,
                             ax, cbg_group='all', mode='ratio', color='slategrey', line_label=None, x_lim=None,
                             set_axis_labels=True):
    assert mode in {'ratio', 'diff', 'percent_change'}
    msa_mc_df = max_cap_df[max_cap_df['MSA_name'] == msa_name]
    max_cap_vals = sorted(msa_mc_df.counterfactual_max_capacity_alpha.unique())
    msa_u_df = uniform_df[uniform_df['MSA_name'] == msa_name]
    uniform_vals = sorted(msa_u_df.counterfactual_full_activity_alpha.unique())
    assert(len(max_cap_vals) == (len(uniform_vals)-1))
    X = []
    Y_mean = []
    Y_lower = []
    Y_upper = []
    for mc_val, u_val in zip(max_cap_vals, uniform_vals[:-1]):
        print('Comparing max_cap_alpha=%.2f to full_activity_alpha=%.2f' % (mc_val, u_val))
        mc_subdf = msa_mc_df[msa_mc_df['counterfactual_max_capacity_alpha'] == mc_val]
        u_subdf = msa_u_df[msa_u_df['counterfactual_full_activity_alpha'] == u_val]
        mc_all_lir = []
        u_all_lir = []
        visits_lost = None
        for baseline_model in mc_subdf.counterfactual_baseline_model.values:  # per param
            curr_df = mc_subdf[mc_subdf['counterfactual_baseline_model'] == baseline_model]
            assert len(curr_df) == 1
            int_lir, fin_lir, visits_lost = get_lir_checkpoints_and_prop_visits_lost(curr_df.iloc[0]['timestring'], 
                                                                                     intervention_idx, 
                                                                                     group=cbg_group,
                                                                                     full_activity_num_visits=full_activity_num_visits)
            frac_gained = fin_lir - int_lir 
            mc_all_lir.extend(list(frac_gained.copy()))  # per seed
            
            curr_df = u_subdf[u_subdf['counterfactual_baseline_model'] == baseline_model]
            assert len(curr_df) == 1
            int_lir, fin_lir, visits_lost = get_lir_checkpoints_and_prop_visits_lost(curr_df.iloc[0]['timestring'], 
                                                                                     intervention_idx,
                                                                                     group=cbg_group)
            frac_gained = fin_lir - int_lir 
            u_all_lir.extend(list(frac_gained.copy()))
        print('Num params * seeds:', len(mc_all_lir))
        if mode == 'ratio':
            curr_comparison = np.array(mc_all_lir) / np.array(u_all_lir)
        elif mode == 'diff':
            curr_comparison = np.array(mc_all_lir) - np.array(u_all_lir)
        else:
            curr_comparison = (np.array(mc_all_lir) - np.array(u_all_lir)) / np.array(u_all_lir)
        X.append(visits_lost)
        Y_mean.append(np.mean(curr_comparison))
        Y_lower.append(np.percentile(curr_comparison, LOWER_PERCENTILE))
        Y_upper.append(np.percentile(curr_comparison, UPPER_PERCENTILE))
    
    if line_label is None:
        ax.plot(X, Y_mean, marker='o', linewidth=2, color=color)
    else:
        ax.plot(X, Y_mean, marker='o', linewidth=2, color=color, label=line_label)
    ax.fill_between(X, Y_lower, Y_upper, alpha=0.2, color=color)
    if x_lim is not None:
        ax.set_xlim(x_lim)
    if mode == 'ratio':
        ax.plot([min(X), max(X)], [1, 1], color='grey', linestyle='--')
        ylabel = 'Ratio of increase from reopening'
    else:
        ax.plot([min(X), max(X)], [0, 0], color='grey', linestyle='--')
        ylabel = 'Difference in increase from reopening'
    if set_axis_labels:
        ax.set_xlabel('Fraction of visits lost\n(compared to full reopening)', fontsize=16)
        ax.set_ylabel(ylabel, fontsize=16)
    ax.grid(alpha=.3)
    ax.tick_params(labelsize=16)
    ax.yaxis.set_major_formatter(tick.FuncFormatter(reformat_decimal_as_percent))
    return X, Y_mean, Y_lower, Y_upper

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
X, Y_mean, Y_lower, Y_upper = plot_pairwise_comparison(max_cap_df, uniform_df, DC_NAME, dc_full_activity, 
                                                      intervention_idx, ax, x_lim=x_lim, mode='percent_change',
                                                      set_axis_labels=False)
print(X)
print(Y_mean)
plt.show()

## Figure 3f: disparities in partial reopening impact

In [ ]:
# we include Figure 3f here with Figure 2c bc they are both analyzing the impacts of the 
# clipping reopening strategy and they share code
msa_name = DC_NAME
full_activity = dc_full_activity
fig_real, ax_real = plt.subplots(figsize=(7, 7))
X, Y_min, Y_mean, Y_max = make_pareto_plot(uniform_df, msa_name, 'counterfactual_full_activity_alpha', intervention_idx,
                 None, cbg_group='all', plot_lir_diff=True)
X, Y_min, Y_mean, Y_max = make_pareto_plot(max_cap_df, msa_name, 'counterfactual_max_capacity_alpha', intervention_idx,
                 ax_real, 'tab:blue', annotate_points=False,
                 full_activity_num_visits=full_activity, line_label='overall', 
                 add_X=X[-1], add_Y=(Y_min[-1], Y_mean[-1], Y_max[-1]),
                 cbg_group='all', set_axis_labels=False, plot_lir_diff=True)
print('Overall X:', X)
print('Overall Y means:', Y_mean)
X, Y_min, Y_mean, Y_max = make_pareto_plot(uniform_df, msa_name, 'counterfactual_full_activity_alpha', intervention_idx,
                 None, cbg_group=LOWINCOME, plot_lir_diff=True)
X, Y_min, Y_mean, Y_max = make_pareto_plot(max_cap_df, msa_name, 'counterfactual_max_capacity_alpha', intervention_idx,
                 ax_real, 'darkorchid', annotation_color='darkorchid',
                 full_activity_num_visits=full_activity, line_label='bottom income decile', 
                 add_X=X[-1], add_Y=(Y_min[-1], Y_mean[-1], Y_max[-1]), 
                 cbg_group=LOWINCOME, set_axis_labels=True, plot_lir_diff=True)
print('Bottom decile Y means:', Y_mean)
plt.show()

## Figure 2d: Reopening different POI categories

In [ ]:
# load dataframe

intervention_results = []
min_timestring = '2020_05_29_18_53_39_321235'
for msa in msas:
    intervention_results.append(evaluate_all_fitted_models_for_msa(msa, 
                                                          min_timestring=min_timestring, 
                                                          key_to_sort_by=None, 
                                                          required_properties=
                                                                   {'experiment_to_run':'test_interventions'}))
intervention_df = pd.concat(intervention_results)
intervention_df['MSA_name'] = intervention_df['data_kwargs'].map(lambda x:x['MSA_name'])
assert (intervention_df['poi_psi'] > 0).all()
print("Total models loaded after filtering for intervention experiments: %i" % len(intervention_df))

for k in ['alpha', 
          'extra_weeks_to_simulate', 
          'intervention_datetime', 
          'top_category', 
          'sub_category']:
    intervention_df['counterfactual_%s' % k] = intervention_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x[k])
intervention_df['model_fit_rank_for_msa'] = intervention_df['counterfactual_poi_opening_experiment_kwargs'].map(lambda x:x['model_quality_dict']['model_fit_rank_for_msa'])

assert len(intervention_df) == 6000
print("Model params")
print(intervention_df.groupby(['MSA_name', 'poi_psi', 'p_sick_at_t0', 'home_beta']).size())


In [ ]:
# get list of categories and make box plots in paper. 

most_visited_poi_subcategories = get_list_of_poi_subcategories_with_most_visits(n_poi_categories=20)




for msa in msas:
    make_boxplot_of_poi_reopening_effects(intervention_df, 
                                          [msa], 
                                          poi_characteristics, 
                                          titlestring=MSAS_TO_PRETTY_NAMES[msa], 
                                          cats_to_plot=most_visited_poi_subcategories, 
                                          filename='covid_figures_for_paper/reopening_impact_boxplots_%s.pdf' % msa)
    plt.show()
    

make_boxplot_of_poi_reopening_effects(intervention_df, 
                                      msas,
                                      poi_characteristics, 
                                      titlestring='All MSAs', 
                                      cats_to_plot=most_visited_poi_subcategories, 
                                      filename='covid_figures_for_paper/reopening_impact_boxplots_all_MSAs.pdf')
plt.show()


In [ ]:
# Disparate impact plots for POI categories (shown in supplement)

plot_reopening_effect_by_poi_category_with_disparate_impact(intervention_df, 
                                      medians_or_deciles='deciles', 
                                      cats_to_plot=most_visited_poi_subcategories,
                                      filename='covid_figures_for_paper/reopening_by_poi_decile_income.pdf')
plt.show()


# Equity analysis

In [ ]:
LOWINCOME = 'median_household_income_bottom_decile'
HIGHINCOME = 'median_household_income_top_decile'
WHITE = 'p_white_top_decile'
NONWHITE = 'p_white_bottom_decile'
PATH_TO_SAVED_ATTRIBUTES = os.path.join('/dfs/scratch1/safegraph_homes/all_aggregate_data/', 'msa_data_for_analysis.pkl')

## Code to compute relevant POI attributes

In [ ]:
# compute and save relevant POI attributes
msa_data_to_save = {}
for msa_name in MSAS:
    df = helper.load_dataframe_for_individual_msa(MSA_name=msa_name)
    data_kwargs = {'MSA_name':msa_name}
    model_kwargs = {'min_datetime':MIN_DATETIME,
                         'max_datetime':MAX_DATETIME,
                         'exogenous_model_kwargs': {
                            'home_beta':1e-2,
                            'poi_psi':1000,
                            'p_sick_at_t0':1e-4,
                            'just_compute_r0':False},
                          'poi_attributes_to_clip':{'clip_areas':True,
                                          'clip_dwell_times':True,
                                          'clip_visits':True},
                          'correct_poi_visits':True}
    # modify fit_ane_save_one_model temporarily to return these things
    out = fit_and_save_one_model(None, model_kwargs, data_kwargs, d=df)
    msa_data_to_save[msa_name] = {'poi_categories':out[0],
                                  'poi_areas':out[1],
                                  'poi_dwell_times':out[2],
                                  'poi_dwell_time_correction_factors':out[3],
                                  'cbg_idx_groups_to_track':out[4],
                                  'cbg_sizes':out[5]}
f = open(PATH_TO_SAVED_ATTRIBUTES, 'wb')
pickle.dump(msa_data_to_save, f)
f.close()

In [ ]:
# can just load the saved attributes without recomputing
f = open(PATH_TO_SAVED_ATTRIBUTES, 'rb')
msa_to_small_stuff = pickle.load(f)
f.close()
print(msa_to_small_stuff.keys())

MSA_TO_STUFF = msa_to_small_stuff
for msa_name in MSAS:
    print(msa_name)
    fn = get_ipf_filename(msa_name, MIN_DATETIME, MAX_DATETIME, True, True)
    print(fn)
    f = open(fn, 'rb')
    poi_cbg_visits_list = pickle.load(f)
    f.close()
    MSA_TO_STUFF[msa_name]['poi_cbg_visits_list'] = poi_cbg_visits_list

In [ ]:
def get_poi_attributes_for_msa(msa_name, poi_time_counts=None, group_to_track=None, verbose=True):
    '''
    Creates a dataframe where each row represents a POI and contains its original
    POI index, static attributes of the POI (top_category, sub_category, area, dwell_time,
    dwell_time_correction_factor), and its time-varying attributes (avg_occupancy, avg_density, 
    avg_transmission_rate) averaged over all hours and, if applicable, with respect to the CBGs 
    specified in group_to_track. 
    '''
    stuff = MSA_TO_STUFF[msa_name]
    poi_areas = stuff['poi_areas']
    poi_dwell_times = stuff['poi_dwell_times']
    poi_dwell_time_correction_factors = stuff['poi_dwell_time_correction_factors']
    poi_categories = stuff['poi_categories']
    poi_categories['pretty_sub_category'] = poi_categories['sub_category'].map(lambda x:SUBCATEGORIES_TO_PRETTY_NAMES[x] if x in SUBCATEGORIES_TO_PRETTY_NAMES else x)
    if poi_time_counts is None:  # use saved poi_cbg_visits_list
        poi_cbg_visits_list = stuff['poi_cbg_visits_list']
        T = len(poi_cbg_visits_list)
        num_pois, num_cbgs = poi_cbg_visits_list[0].shape
    else:
        num_pois, T = poi_time_counts.shape
        num_cbgs = None
        poi_cbg_visits_list = None
        assert group_to_track is None  # can't differentiate CBGs if we only have overall POI visit counts
    if group_to_track is None:
        cbg_idx = None
    else:
        cbg_idx = stuff['cbg_idx_groups_to_track'][group_to_track]
    
    print('Aggregating data from %d hours' % T)
    static_factors = poi_dwell_time_correction_factors / poi_areas
    total_weighted_transmission_rates = np.zeros(num_pois)
    total_weighted_occupancy = np.zeros(num_pois)
    total_weighted_density = np.zeros(num_pois) 
    total_poi_visits = np.zeros(num_pois)  # denominator: only visits from CBGs of interest
    
    for t in range(T):
        if poi_time_counts is None:
            if cbg_idx is None:
                indicator = np.ones(num_cbgs)
            else:
                indicator = np.zeros(num_cbgs)
                indicator[cbg_idx] = 1.0
            poi_cbg_visits = poi_cbg_visits_list[t]
            poi_visits_from_any_cbg = poi_cbg_visits @ np.ones(num_cbgs)
            poi_visits_from_cbgs_of_interest = poi_cbg_visits @ indicator
        else:
            poi_visits_from_any_cbg = poi_time_counts[:, t]
            poi_visits_from_cbgs_of_interest = poi_visits_from_any_cbg
        
        poi_densities = poi_visits_from_any_cbg / poi_areas 
        poi_transmission_rates = poi_visits_from_any_cbg * static_factors
        total_weighted_transmission_rates = total_weighted_transmission_rates + (poi_visits_from_cbgs_of_interest * poi_transmission_rates)
        total_weighted_occupancy = total_weighted_occupancy + (poi_visits_from_cbgs_of_interest * poi_visits_from_any_cbg)
        total_weighted_density = total_weighted_density + (poi_visits_from_cbgs_of_interest * poi_densities)
        total_poi_visits = total_poi_visits + poi_visits_from_cbgs_of_interest

    kept_idx = np.array(range(num_pois))[total_poi_visits >= 1]  # only want pois with at least 1 visits
    if verbose:
        print('Dropped %d/%d POIs with 0 visits in this time period.' % (num_pois - len(kept_idx), num_pois))
    poi_attributes_df = pd.DataFrame.from_dict({'poi_idx':kept_idx, 
                                         'top_category':poi_categories.top_category.values[kept_idx],
                                         'sub_category':poi_categories.sub_category.values[kept_idx],
                                         'pretty_sub_category':poi_categories.pretty_sub_category.values[kept_idx],
                                         'total_num_visits':total_poi_visits[kept_idx],
                                         'area':poi_areas[kept_idx].copy(), 
                                         'dwell_time':poi_dwell_times[kept_idx].copy(),
                                         'dwell_time_correction_factor':poi_dwell_time_correction_factors[kept_idx].copy(),
                                         'avg_occupancy':total_weighted_occupancy[kept_idx] / total_poi_visits[kept_idx],
                                         'avg_density':total_weighted_density[kept_idx] / total_poi_visits[kept_idx],
                                         'avg_transmission_rate':total_weighted_transmission_rates[kept_idx] / total_poi_visits[kept_idx],
                                         })
    return poi_attributes_df

In [ ]:
def get_category_attributes_from_poi_attributes(poi_attributes_df, categories, pop_size=None, verbose=True):
    category_num_visits = []
    category_avg_areas = []  # weighted average over POIs
    category_median_areas = []  # median (no weighting) over POIs
    category_avg_dwell_times = []
    category_median_dwell_times = []
    category_avg_occupancies = []
    category_median_occupancies = []
    category_avg_densities = []
    category_median_densities = []
    category_avg_transmission_rates = []
    category_median_transmission_rates = []
    for cat in categories:
        subdf = poi_attributes_df[poi_attributes_df['pretty_sub_category'] == cat]
        if len(subdf) > 0:
            num_visits_per_poi = subdf['total_num_visits'].values
            category_num_visits.append(np.sum(num_visits_per_poi))
            prop_visits_per_poi = num_visits_per_poi / np.sum(num_visits_per_poi)
            attributes_per_poi = subdf[['area', 'dwell_time_correction_factor', 'avg_occupancy',
                                        'avg_density', 'avg_transmission_rate']].values
            attribute_averages = (prop_visits_per_poi @ attributes_per_poi).T
            attribute_medians = np.median(attributes_per_poi, axis=0)  # median over POIs
        else:
            if verbose:
                print('Missing visits to any POI in %s' % cat)
            category_num_visits.append(0)
            attribute_averages = np.ones(5) * np.nan
            attribute_medians = np.ones(5) * np.nan
        category_avg_areas.append(attribute_averages[0])
        category_median_areas.append(attribute_medians[0])
        category_avg_dwell_times.append(attribute_averages[1])
        category_median_dwell_times.append(attribute_medians[1])
        category_avg_occupancies.append(attribute_averages[2])
        category_median_occupancies.append(attribute_medians[2])
        category_avg_densities.append(attribute_averages[3])
        category_median_densities.append(attribute_medians[3])
        category_avg_transmission_rates.append(attribute_averages[4])
        category_median_transmission_rates.append(attribute_medians[4])
    category_attributes_df = pd.DataFrame.from_dict({'category':categories,
                                                     'total_num_visits':category_num_visits,
                                                     'avg_area':category_avg_areas,
                                                     'median_area':category_median_areas,
                                                     'avg_dwell_time_correction_factor':category_avg_dwell_times,
                                                     'median_dwell_time_correction_factor':category_median_dwell_times,
                                                     'avg_occupancy':category_avg_occupancies,
                                                     'median_occupancy':category_median_occupancies,
                                                     'avg_density':category_avg_densities,
                                                     'median_density':category_median_densities,
                                                     'avg_transmission_rate':category_avg_transmission_rates,
                                                     'median_transmission_rate':category_median_transmission_rates})
    if pop_size is not None:
        category_attributes_df['num_visits_per_capita'] = category_attributes_df.total_num_visits.values / pop_size
    return category_attributes_df

In [ ]:
TOP_CATEGORIES = get_list_of_poi_subcategories_with_most_visits(n_poi_categories=20)
PRETTY_TOP_CATEGORIES = [SUBCATEGORIES_TO_PRETTY_NAMES[cat] for cat in TOP_CATEGORIES]

## Figure 3a/b: disparate impact plots 

In [ ]:
def get_LIR_ratios_from_models(timestrings, g1, g2):
    """
    Returns g1/g2. One row for each model, one column for 
    """
    LIR_ratios = []
    for timestring in timestrings:
        mdl, _, _, _, _ = load_model_and_data_from_timestring(
            timestring,
            load_fast_results_only=False, 
            load_full_model=True)

        ratios = []
        for idx, g in enumerate([g1, g2]):        
            n_group = mdl.history[g]['total_pop']
            n_LIR = (mdl.history[g]['latent'] + mdl.history[g]['infected'] + mdl.history[g]['removed'])[:, -1]
            ratios.append(n_LIR / n_group)
            


        LIR_ratios.append(ratios[0] / ratios[1])    
    LIR_ratios = np.array(LIR_ratios)
    assert len(LIR_ratios) == len(timestrings)
    return LIR_ratios

SES_results = []
all_ses_ratios = []
sort_key_for_ses_analysis = 'loss_dict_daily_cases_RMSE'
print("Using parameters MAX_MODELS_TO_TAKE_PER_MSA=%i and ACCEPTABLE_LOSS_TOLERANCE=%2.3f" % 
      (MAX_MODELS_TO_TAKE_PER_MSA, ACCEPTABLE_LOSS_TOLERANCE))
for city in sorted(list(set(non_ablation_df['MSA_name']))):
    
    city_df = non_ablation_df.loc[(non_ablation_df['MSA_name'] == city)]
    min_loss = city_df[sort_key_for_ses_analysis].min()
    city_df = (city_df.loc[city_df[sort_key_for_ses_analysis] <= (min_loss * ACCEPTABLE_LOSS_TOLERANCE)]
               .sort_values(by=sort_key_for_ses_analysis)
               .iloc[:MAX_MODELS_TO_TAKE_PER_MSA])
    max_ratio = city_df[sort_key_for_ses_analysis].max() / city_df[sort_key_for_ses_analysis].min()
    assert max_ratio > 1 and max_ratio < ACCEPTABLE_LOSS_TOLERANCE
    print("Plotting %i models for %s" % (len(city_df), city))
    for k in ['p_black', 'median_household_income', 'p_white']:

        for comparison in ['medians', 'deciles']:
            if comparison == 'medians':
                disadvantaged_ratios = get_LIR_ratios_from_models(
                        city_df['timestring'], 
                        f'{k}_below_median',
                        f'{k}_above_median')

            else:
                disadvantaged_ratios = get_LIR_ratios_from_models(
                        city_df['timestring'], 
                        f'{k}_bottom_decile',
                        f'{k}_top_decile')
                    
            if k == 'p_black':
                disadvantaged_ratios = 1 / disadvantaged_ratios
                
            # disadvantaged ratios here is n_models x n_seeds
            SES_results.append({"city":city, 
                                'demo':k, 
                                'comparison':comparison, 
                                'median_ratio':np.median(disadvantaged_ratios),
                                'disadvantaged_group_is_more_sick':np.mean(disadvantaged_ratios > 1), 
                               'n':len(city_df)})
            all_ses_ratios.append(pd.DataFrame({'city':city, 
                                                'ratio':disadvantaged_ratios.flatten(),
                                                'comparison':comparison, 
                                                'demo':k}))

    #SES_results.append({"city":city, 'demo':k, 'comparison':'medians', val:above_median_larger})
all_ses_ratios = pd.concat(all_ses_ratios)
all_ses_ratios['MSA'] = all_ses_ratios['city'].map(lambda x:MSAS_TO_PRETTY_NAMES[x])
SES_results = pd.DataFrame(SES_results)
SES_results.loc[(SES_results['demo'] == 'median_household_income')].sort_values(by=['city', 'comparison'])



In [ ]:
# make plot for paper
for comparison in ['deciles', 'medians']:
    for demographic in ['p_white', 'median_household_income']:
        make_ses_disparities_infection_ratio_plot_for_paper(all_ses_ratios, 
                                                    comparisons=[comparison], 
                                                    demographic=demographic, 
                                                    filename='covid_figures_for_paper/infection_rate_disparities_%s_%s.pdf' 
                                                            % (demographic, comparison))

    

## Figure 3c: disparate impact within POI categories

In [ ]:
# load best fit models
best_models = []
min_timestring = '2020_05_30_14_28_28_220347'
max_timestring = '2020_06_16'
required_properties = {'experiment_to_run':'rerun_best_models_and_save_cases_per_poi'}
for msa in MSAS:
    best_models.append(evaluate_all_fitted_models_for_msa(msa, 
                                                      min_timestring=min_timestring,
                                                      max_timestring=max_timestring,
                                                      required_properties=required_properties,
                                                      key_to_sort_by=None))
best_models_df = pd.concat(best_models)
best_models_df['MSA_name'] = best_models_df['data_kwargs'].map(lambda x:x['MSA_name'])
print("Total models loaded: %i" % len(best_models_df))

In [ ]:
def get_frac_infections_at_each_category_for_groups(timestring, groups, categories_to_plot, poi_categories):
    _, _, _, model_results, fast_to_load_results = load_model_and_data_from_timestring(timestring, 
                                                                                       load_fast_results_only=False,
                                                                                       load_full_model=False)
    group2fracs = {group:[] for group in groups}
    for group in groups:
        num_cases_per_poi = model_results['history'][group]['num_cases_per_poi']
        n_seeds, n_pois = num_cases_per_poi.shape
        assert n_pois == len(poi_categories)
        pop_size = model_results['history'][group]['total_pop']
        for s in range(n_seeds):
            frac_pop_infected = []
            for cat in categories_to_plot:
                cat_idx = poi_categories == cat 
                num_cases_at_cat = np.sum(num_cases_per_poi[s][cat_idx])
                frac_pop_infected.append(num_cases_at_cat / pop_size)
            group2fracs[group].append(frac_pop_infected)
    return group2fracs

In [ ]:
def plot_frac_infected_per_category_for_multiple_models(results_df, msa_name, ax, num_categories=20):
    categories_to_plot = PRETTY_TOP_CATEGORIES[:num_categories]
    msa_df = results_df[results_df['MSA_name'] == msa_name]
    poi_categories = MSA_TO_STUFF[msa_name]['poi_categories']
    poi_categories['pretty_sub_category'] = poi_categories['sub_category'].map(lambda x:SUBCATEGORIES_TO_PRETTY_NAMES[x] if x in SUBCATEGORIES_TO_PRETTY_NAMES else x)
    bottom_decile_fracs = []  # num_models x num_categories
    top_decile_fracs = []  # num_models x num_categories
    for ts in msa_df.timestring.values:
        group2fracs = get_frac_infections_at_each_category_for_groups(ts, [LOWINCOME, HIGHINCOME], categories_to_plot, poi_categories.pretty_sub_category.values)
        bottom_decile_fracs.extend(group2fracs[LOWINCOME])
        top_decile_fracs.extend(group2fracs[HIGHINCOME])
    
    print('Num params * seeds:', len(bottom_decile_fracs))
    bottom_decile_fracs = np.array(bottom_decile_fracs)
    bottom_decile_mean = INCIDENCE_POP * np.mean(bottom_decile_fracs, axis=0)
    bottom_decile_min = INCIDENCE_POP * np.percentile(bottom_decile_fracs, LOWER_PERCENTILE, axis=0)
    bottom_decile_max = INCIDENCE_POP * np.percentile(bottom_decile_fracs, UPPER_PERCENTILE, axis=0)    
    top_decile_fracs = np.array(top_decile_fracs)
    top_decile_mean = INCIDENCE_POP * np.mean(top_decile_fracs, axis=0)
    top_decile_min = INCIDENCE_POP * np.percentile(top_decile_fracs, LOWER_PERCENTILE, axis=0)
    top_decile_max = INCIDENCE_POP * np.percentile(top_decile_fracs, UPPER_PERCENTILE, axis=0)

    y_pos = range(num_categories)
    sorted_idx = np.argsort(bottom_decile_mean)  # sort category plotting by their impact
    ax.plot(bottom_decile_mean[sorted_idx], y_pos, label='bottom income decile', color='darkorchid', linewidth=2)
    ax.fill_betweenx(y=y_pos, x1=bottom_decile_min[sorted_idx],
                     x2=bottom_decile_max[sorted_idx], color='darkorchid', alpha=.3)
    ax.plot(top_decile_mean[sorted_idx], y_pos, label='top income decile', color='darkgoldenrod', linewidth=2)
    ax.fill_betweenx(y=y_pos, x1=top_decile_min[sorted_idx],
                     x2=top_decile_max[sorted_idx], color='darkgoldenrod', alpha=.5)
    ax.set_yticks(y_pos)
    labels = [categories_to_plot[i] for i in sorted_idx]
    ax.set_yticklabels(labels, fontsize=16)
    ax.tick_params(axis='x', labelsize=16)
    ax.set_xlabel('Cumulative infections (per 100k) at category', fontsize=16)
    curr_lim = ax.get_xlim()
    ax.set_xlim(0, curr_lim[1])
    ax.grid(alpha=.3)
    return labels

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
category_order = plot_frac_infected_per_category_for_multiple_models(best_models_df, DC_NAME, ax)
ax.legend(fontsize=16, loc='lower right')
plt.show()

## Figure 3d: mobility over time

In [ ]:
def make_mobility_comparison_line_plot(msa_name, min_datetime, max_datetime, 
                                       group1, group1_label, group1_color, 
                                       group2, group2_label, group2_color, 
                                       ax, num_hours_to_agg=24, set_labels=True):
    stuff = MSA_TO_STUFF[msa_name]
    cbg_idx_groups_to_track, cbg_sizes = stuff['cbg_idx_groups_to_track'], stuff['cbg_sizes']
    poi_cbg_visits_list = stuff['poi_cbg_visits_list']    
    hours = helper.list_hours_in_range(min_datetime, max_datetime)
    assert (len(hours) % num_hours_to_agg) == 0
    hours_to_plot = [hours[t] for t in np.arange(0, len(hours), num_hours_to_agg)]
    num_pois, num_cbgs = poi_cbg_visits_list[0].shape
    indicator_1 = np.zeros(num_cbgs)
    indicator_1[cbg_idx_groups_to_track[group1]] = 1.0
    pop_size_1 = np.sum(cbg_sizes[cbg_idx_groups_to_track[group1]])
    indicator_2 = np.zeros(num_cbgs)
    indicator_2[cbg_idx_groups_to_track[group2]] = 1.0
    pop_size_2 = np.sum(cbg_sizes[cbg_idx_groups_to_track[group2]])
    Y1 = []
    Y2 = []
    for t in range(len(hours_to_plot)):
        total_1 = 0
        total_2 = 0
        start_hr = t*num_hours_to_agg
        for hr in range(start_hr, start_hr+num_hours_to_agg):
            total_1 += np.sum(poi_cbg_visits_list[hr] @ indicator_1)
            total_2 += np.sum(poi_cbg_visits_list[hr] @ indicator_2)
        Y1.append(total_1 / pop_size_1)  # per-capita num visits over time period
        Y2.append(total_2 / pop_size_2)
    ax.plot_date(hours_to_plot, Y1, linestyle='-', marker='.', linewidth=2, label=group1_label, color=group1_color)
    ax.plot_date(hours_to_plot, Y2, linestyle='-', marker='.', linewidth=2, label=group2_label, color=group2_color)
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(mdates.SU, interval=2))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    ax.tick_params(labelsize=16)
    ax.grid(alpha=0.2)
    
    if set_labels:
        ax.set_xlabel('Date', fontsize=16)
        ax.set_ylabel('Per capita mobility', fontsize=16)
        ax.set_title(MSAS_TO_PRETTY_NAMES[msa_name], fontsize=18)
        ax.legend(fontsize=16)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
make_mobility_comparison_line_plot(DC_NAME, MIN_DATETIME, MAX_DATETIME, 
                                   LOWINCOME, 'bottom income decile', 'darkorchid',
                                   HIGHINCOME, 'top income decile', 'darkgoldenrod', ax)

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 20))
plt.subplots_adjust(hspace=0.3)
axes = [ax for axes_row in axes for ax in axes_row]
for i, (ax, msa_name) in enumerate(list(zip(axes, MSAS))):
    print(msa_name)
    ax.set_title(MSAS_TO_PRETTY_NAMES[msa_name], fontsize=14)
    make_mobility_comparison_line_plot(msa_name, MIN_DATETIME, MAX_DATETIME, 
                                       LOWINCOME, 'bottom income decile', 'darkorchid',
                                       HIGHINCOME, 'top income decile', 'darkgoldenrod', 
                                       ax, set_labels=False)
    ax.set_title(MSAS_TO_PRETTY_NAMES[msa_name], fontsize=18)
    if i == 0:
        ax.legend(bbox_to_anchor=(-0.5, 2), fontsize=16)
plt.show()

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 20))
plt.subplots_adjust(hspace=0.3)
axes = [ax for axes_row in axes for ax in axes_row]
for i, (ax, msa_name) in enumerate(list(zip(axes, MSAS))):
    print(msa_name)
    ax.set_title(MSAS_TO_PRETTY_NAMES[msa_name], fontsize=14)
    make_mobility_comparison_line_plot(msa_name, MIN_DATETIME, MAX_DATETIME, 
                                       NONWHITE, 'bottom decile (% white)', 'forestgreen',
                                       WHITE, 'top decile (% white)', 'salmon', 
                                       ax, set_labels=False)
    ax.set_title(MSAS_TO_PRETTY_NAMES[msa_name], fontsize=18)
    if i == 0:
        ax.legend(bbox_to_anchor=(-0.5, 2), fontsize=16)
plt.show()

## Fig 3e: transmission rates per category

In [ ]:
def make_category_comparison_scatter_plot(attributes_df_1, attributes_df_2,
                                          attribute_to_plot, ax, color, title,
                                          xlabel, ylabel, categories_to_label, 
                                          x_lim=None, y_lim=None, plot_log=False,
                                          psi=1):

    categories = attributes_df_1.category.values
    vals_1 = attributes_df_1[attribute_to_plot].values
    visits_1 = attributes_df_1['num_visits_per_capita'].values
    vals_2 = attributes_df_2[attribute_to_plot].values
    visits_2 = attributes_df_2['num_visits_per_capita'].values
    avg_visits = (visits_1 + visits_2) / 2

    X = vals_2
    Y = vals_1
    if 'transmission_rate' in attribute_to_plot:
        X = X * psi
        Y = Y * psi
    sizes = 200 * avg_visits
    if plot_log:
        X = np.log(X)
        Y = np.log(Y)
        xlabel += ' (log)'
        ylabel += ' (log)'
    ax.scatter(X, Y, s=sizes, alpha=0.6, color=color)
    
    max_val = max(max(X), max(Y))  # set x and y axes to have the same tick ranges
    offset = .1 * max_val
    if x_lim is None:
        ax.set_xlim(0 - offset, max_val + offset)
    else:
        ax.set_xlim(x_lim[0], x_lim[1])
    if y_lim is None:
        ax.set_ylim(0 - offset, max_val + offset)
    else:
        ax.set_ylim(y_lim[0], y_lim[1])
    ax.plot(list(ax.get_xlim()), list(ax.get_xlim()), color='grey', alpha=0.5, label='y=x')
    
    ax.tick_params(labelsize=16)
    ax.grid(alpha=.2)
    ax.legend(fontsize=16)
    ax.set_title(title, fontsize=20)
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)  
    
    for i in range(len(X)):
        cat = categories[i]
        if cat in categories_to_label:
            if attribute_to_plot == 'avg_transmission_rate' and 'Washington' in title:
            # special code to format Washington DC plot
                if cat in {'Fitness Centers', 'Offices of Physicians', 'Hotels & Motels'}:
                    ax.annotate(cat, xy=(X[i], Y[i]), xytext=(-100, 0), textcoords='offset pixels', fontsize=14)
                elif cat in {'Full-Service Restaurants', 'Religious Organizations'}:
                    ax.annotate(cat, xy=(X[i], Y[i]), xytext=(-150, 0), textcoords='offset pixels', fontsize=14)
                elif cat in {'Convenience Stores', 'Department Stores', 'Other General Stores', 
                             'Used Merchandise Stores'}:
                    ax.annotate(cat, xy=(X[i], Y[i]), xytext=(-10, -5), textcoords='offset pixels', fontsize=14)
                elif cat in {'Gas Stations', 'Grocery Stores'}:
                    ax.annotate(cat, xy=(X[i], Y[i]), xytext=(-50, 5), textcoords='offset pixels', fontsize=14)
                else:
                    ax.annotate(cat, xy=(X[i], Y[i]), xytext=(-10, 0), textcoords='offset pixels', fontsize=14)
            else:
                ax.annotate(cat, xy=(X[i], Y[i]), xytext=(-10, 0), textcoords='offset pixels', fontsize=14)

In [ ]:
def make_transmission_rate_scatter_plot_for_paper(save_fig=False):
    categories_to_plot = PRETTY_TOP_CATEGORIES[:12]
    categories_to_label = categories_to_plot[:12]
    stuff = MSA_TO_STUFF[DC_NAME]
    cbg_idx_groups_to_track = stuff['cbg_idx_groups_to_track']
    cbg_sizes = stuff['cbg_sizes']

    li_poi_attr = get_poi_attributes_for_msa(DC_NAME, group_to_track=LOWINCOME)
    li_pop_size = np.sum(cbg_sizes[cbg_idx_groups_to_track[LOWINCOME]])
    li_cat_attr = get_category_attributes_from_poi_attributes(li_poi_attr, categories_to_plot, pop_size=li_pop_size)
    hi_poi_attr = get_poi_attributes_for_msa(DC_NAME, group_to_track=HIGHINCOME)
    hi_pop_size = np.sum(cbg_sizes[cbg_idx_groups_to_track[HIGHINCOME]])
    hi_cat_attr = get_category_attributes_from_poi_attributes(hi_poi_attr, categories_to_plot, pop_size=hi_pop_size)

    fig, ax = plt.subplots(figsize=(7, 7))
    attribute = 'avg_transmission_rate'
    title = '%s: comparison of transmission rates' % MSAS_TO_PRETTY_NAMES[DC_NAME]
    xlabel = 'Avg transmission rate,\ntop income decile'
    ylabel = 'Avg transmission rate,\nbottom income decile'
    make_category_comparison_scatter_plot(li_cat_attr, hi_cat_attr, 
                                          attribute, ax, 'coral', title,
                                          xlabel, ylabel, categories_to_label, psi=4500)

In [ ]:
make_transmission_rate_scatter_plot_for_paper()

## Supplement: transmission rate ratio tables

In [ ]:
def get_attribute_ratios_for_all_msas(group1, group2, num_categories=20):
    categories_to_plot = PRETTY_TOP_CATEGORIES[:num_categories]
    attributes = ['avg_area', 'avg_dwell_time_correction_factor', 'avg_occupancy', 'avg_density', 'avg_transmission_rate']
    col_names = ['%s_ratio' % a for a in attributes]
    all_results = []
    for msa_name in MSAS:
        print('Getting results for', msa_name)
        poi_attr_1 = get_poi_attributes_for_msa(msa_name, group_to_track=group1)
        cat_attr_1 = get_category_attributes_from_poi_attributes(poi_attr_1, categories_to_plot)
        poi_attr_2 = get_poi_attributes_for_msa(msa_name, group_to_track=group2)
        cat_attr_2 = get_category_attributes_from_poi_attributes(poi_attr_2, categories_to_plot)
        df = pd.DataFrame(cat_attr_1[attributes].values / cat_attr_2[attributes].values, columns=col_names)
        df['category'] = categories_to_plot
        df['MSA_name'] = msa_name
        all_results.append(df)
    all_results = pd.concat(all_results)
    return all_results

In [ ]:
all_ses_results = get_attribute_ratios_for_all_msas(LOWINCOME, HIGHINCOME)

In [ ]:
# make summary dataframe of avg_transmission_rate ratios for MSA x category
categories_to_plot = PRETTY_TOP_CATEGORIES[:20]
mat = np.zeros((len(categories_to_plot), len(MSAS)))
for i, cat in enumerate(categories_to_plot):
    subdf = all_ses_results[all_ses_results['category'] == cat]
    for j, msa_name in enumerate(MSAS):
        results = subdf[subdf['MSA_name'] == msa_name]
        assert len(results) == 1
        mat[i, j] = round(results['avg_transmission_rate_ratio'], 3)

# median ratio per msa, aggregated over categories
msa_medians = np.round(np.median(mat, axis=0), 3)
for msa, median in zip(MSAS, msa_medians):
    print(msa, median)

cols = [MSAS_TO_PRETTY_NAMES[m] for m in MSAS]
summary_df = pd.DataFrame(mat, columns=cols, index=categories_to_plot)
summary_df['Med'] = np.round(np.median(summary_df.values, axis=1), 3)
summary_df

In [ ]:
all_race_results = get_attribute_ratios_for_all_msas(NONWHITE, WHITE)

In [ ]:
# create category x MSA df of ratios of avg transmission rates
categories_to_plot = PRETTY_TOP_CATEGORIES[:20]
msas = all_race_results.MSA_name.unique()
mat = np.zeros((len(categories_to_plot), len(msas)))
for i, cat in enumerate(categories_to_plot):
    subdf = all_race_results[all_race_results['category'] == cat]
    for j, msa_name in enumerate(msas):
        results = subdf[subdf['MSA_name'] == msa_name]
        assert len(results) == 1
        mat[i, j] = round(results['avg_transmission_rate_ratio'], 3)

# median ratio per msa, aggregated over categories
msa_medians = np.round(np.median(mat, axis=0), 3)
for msa, median in zip(MSAS, msa_medians):
    print(msa, median)

cols = [MSAS_TO_PRETTY_NAMES[m] for m in msas]
summary_df = pd.DataFrame(mat, columns=cols, index=categories_to_plot)
summary_df['Med'] = np.round(np.median(summary_df.values, axis=1), 3)
summary_df

## Supplement: num visits per capita to POI categories

In [ ]:
def plot_per_capita_category_visits(msa_name, group1, group1_label, group1_color, 
                                    group2, group2_label, group2_color, 
                                    ax, num_categories=20):
    categories_to_plot = PRETTY_TOP_CATEGORIES[:num_categories]
    cbg_sizes = MSA_TO_STUFF[msa_name]['cbg_sizes']
    cbg_idx_groups_to_track = MSA_TO_STUFF[msa_name]['cbg_idx_groups_to_track']
    poi_attr_1 = get_poi_attributes_for_msa(msa_name, group_to_track=group1)
    group1_pop_size = np.sum(cbg_sizes[cbg_idx_groups_to_track[group1]])
    cat_attr_1 = get_category_attributes_from_poi_attributes(poi_attr_1, categories_to_plot, pop_size=group1_pop_size)
    X1 = cat_attr_1.num_visits_per_capita.values
    
    poi_attr_2 = get_poi_attributes_for_msa(msa_name, group_to_track=group2)
    group2_pop_size = np.sum(cbg_sizes[cbg_idx_groups_to_track[group2]])
    cat_attr_2 = get_category_attributes_from_poi_attributes(poi_attr_2, categories_to_plot, pop_size=group2_pop_size)
    X2 = cat_attr_2.num_visits_per_capita.values
    
    bar_pos = np.arange(0, 3*num_categories, 3)
    ax.barh(bar_pos-1, X1, align='center', label=group1_label, color=group1_color)
    ax.barh(bar_pos, X2, align='center', label=group2_label, color=group2_color)
    ax.set_yticks(bar_pos-.5)
    ax.set_yticklabels(categories_to_plot, fontsize=16)
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.tick_params(axis='x', labelsize=16)        
    ax.set_xlabel('Per capita visits to category', fontsize=16)
    ax.legend(fontsize=16, loc='lower right')
    ax.set_title(MSAS_TO_PRETTY_NAMES[msa_name], fontsize=20)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
plot_per_capita_category_visits(DC_NAME, LOWINCOME, 'bottom income decile', 'darkorchid',
                                HIGHINCOME, 'top income decile', 'darkgoldenrod', ax)